In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../BBTRE_physics_analysis/") 

In [4]:
import numpy as np
import xarray as xr
import pandas as pd

from xmitgcm import open_mdsdataset 
from xhistogram.xarray import histogram

from osse import *
from canyon_utils import *
from sloped_MITgcm import *
from wmt import *

### Configuration parameters

In [5]:
# Constant parameters

Γ = 0.0008613659531090722
N = np.sqrt(g*α*Γ)
θ = 1.26E-3
f = -5.3e-5

h = 230
k0 = 5.2e-5
k1 = 1.8e-3

dx = 600.; dy = 600.
delta_t = 15.

### Load output

In [6]:
runname = "run"
data_dir = f"../../MITgcm/experiments/rotated_BBTRE_canyon-tracer/{runname}/"
budget_diags = ["budgetDiag", "tracer01Diag", "tracer02Diag", "tracer03Diag"]
ds = open_mdsdataset(data_dir,ignore_unknown_vars=True,prefix=budget_diags, delta_t=delta_t)
ds = ds.drop(['maskC', 'TOTTTEND', 'Tp_gTr01', 'Tp_gTr02', 'Tp_gTr03'])
ds = ds.sel(time=ds['time'][1::])
Lx = dx*ds["XC"].size
ds = periodic_extend(ds, ['XC', 'XG'], Lx, [-1,0])
xslice = [500, 500+800]
ds = ds.isel(XC=slice(*xslice), XG=slice(*xslice))
ds, grid = add_rotated_coords(ds, θ)

ds = ds.drop_dims(['XG', 'YG', 'Zp1', 'Zu', 'Zl'])
ds['dV'] = (ds.drF * ds.rA * ds.hFacC)

ds = ds.assign_coords({'days': (ds['time'].astype('float64')*1.e-9/86400.) - 5000.*(delta_t/60.)})

In [44]:
ids_tmp = ds.sel(YC=ds.YC[ds.YC.size//2]).sel(XC=ds.XC[50::100])

In [46]:
# from dask.diagnostics import ProgressBar
# with ProgressBar():
ids = xr.concat([ids_tmp.sel(time=t).compute() for t in ds.time], dim='time')

### Reconstructing the full temperature field

In [47]:
ids['THETA'] = ids['THETA'].where(ids['THETA'] != 0.)
ids['THETA_BG_C'] = Γ*ids['Zr']
ids['θ'] = ids['THETA'] + ids['THETA_BG_C']

## Various first moments

In [48]:
for tr in [1, 2, 3]:
    print(f"Computing various 1st moments for Tracer {tr}")
    ids[f'TRAC0{tr}dV'] = (ids[f'TRAC0{tr}']*ids['dV']).compute()
    ids[f'M_Tr0{tr}'] = ids[f'TRAC0{tr}dV'].sum(dim=['Z', 'XC']).compute()
    ids[f'Xbar_Tr0{tr}'] = ((ids['XC']*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()
    ids[f'Zbar_Tr0{tr}'] = ((ids['Z']*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()
    ids[f'Zrbar_Tr0{tr}'] = ((ids['Zr']*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()
    #ids[f'HABbar_Tr0{tr}'] = ((ids['Z_habC']*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()
    ids[f'θbar_Tr0{tr}'] = ((ids['θ']*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()
    ids[f'θ2bar_Tr0{tr}'] = (((ids['θ']**2)*ids[f'TRAC0{tr}dV']).sum(dim=['Z', 'XC']) / ids[f'M_Tr0{tr}']).compute()

Computing various 1st moments for Tracer 1
Computing various 1st moments for Tracer 2
Computing various 1st moments for Tracer 3


In [50]:
saving = ids.copy()

saving['time'].dims
for dv in list(saving.data_vars)+list(saving.coords):
    if (saving[dv].dims != ('time',)):
        saving = saving.drop_vars([dv])
        
saving.to_netcdf("../../data/BBTRE-tracer/subsampled_tracer_moments.nc", mode='w')